# Calculate scalar QNMs of RN AdS black hole using the ultraspherical spectral method

We are interested in scalar perturbations of a Reissner-Nordström-AdS (RNAdS) black hole metric:

$$ds^2 = f(r)dt^2 - \frac{dr^2}{f(r)} - r^2(d\theta^2 + \sin^2\theta d\phi^2),$$ 

where $f(r) = \Delta/r^2$ and $\Delta = r^2 - 2Mr + Q^2 + r^4/R^2$. Here, $M$ denotes the black hole mass, $Q$ the charge, and $R$ the AdS radius.

## References

- [[gr-qc/0301052] Quasinormal modes of Reissner-Nordström-anti-de Sitter black holes: scalar, electromagnetic and gravitational perturbations](https://arxiv.org/abs/gr-qc/0301052)
- [[2209.09324] Calculating quasinormal modes of Schwarzschild anti-de Sitter black holes using the continued fraction method](https://arxiv.org/abs/2209.09324)
- [[hep-th/0003295] Quasinormal modes of Reissner-Nordström Anti-de Sitter Black Holes](https://arxiv.org/abs/hep-th/0003295)
- [[1202.1347] A fast and well-conditioned spectral method](https://arxiv.org/abs/1202.1347)

In [4]:
using ApproxFun, LinearAlgebra, Markdown

We use `Float64` by default to obtain high accuracy results. For even higher precision, `Double64` (requires the `DoubleFloats.jl` package), `BigFloat`, or other types can be used.

In [6]:
TF = Float64

Float64

We choose units such that $R = 1$. The extremal value of the black hole charge, $Q_{\text{ext}}$, is given by the following function of the hole's horizon radius:

$$Q_{\text{ext}}^2 = r_{+}^2 \left(1 + \frac{3 r_{+}^2}{R^2}\right) \,.$$

The black hole mass relates to its charge $Q$ and horizon radius $r_+$ via:  

$$M = \frac{1}{2}\left(r_+ + \frac{r_+^3}{R^2} + \frac{Q^2}{r_+}\right).$$

In [37]:
# AdS Radius
R = one(TF)
# horizon radius
r₊ = parse(TF, "1.0")
# black hole charge
Q = parse(TF, "0.0")

M = (r₊ + r₊^3 / R^2 + Q^2 / r₊) / 2
Qext = r₊ * sqrt(1 + 3 * r₊^2 / R^2)

if Q < Qext
    Markdown.parse("Black Hole Parameters:\n\$R = $R,\\, r_+ = $r₊,\\, M = $M,\\, Q = $Q,\\, Q_\\text{ext} = $Qext,\\, Q / Q_\\text{ext} = $(Q / Qext)\$")
else
    # Q is out of range
    Markdown.parse("Black Hole Parameters: Q is out of range!\n\$R = $R,\\, r_+ = $r₊,\\, M = $M,\\, Q = $Q,\\, Q_\\text{ext} = $Qext,\\, Q / Q_\\text{ext} = $(Q / Qext)\$")
end

Black Hole Parameters: $R = 1.0,\, r_+ = 1.0,\, M = 1.0,\, Q = 0.0,\, Q_\text{ext} = 2.0,\, Q / Q_\text{ext} = 0.0$


The radial part of scalar perturbations satisfies the wave equation:  

$$\frac{d^2 \psi}{dr_*^2} + \left(\omega^2 - V\right)\psi = 0,$$  

with potential:

$$V = f(r)\left[\frac{l(l+1)}{r^2} + \frac{f'(r)}{r}\right].$$  

Boundary conditions enforce ingoing waves at the horizon and no outgoing waves at infinity:

$$\psi(r) \approx \begin{cases}  
e^{-i\omega r_*} & \text{as } r \to r_+, \\  
0 & \text{as } r \to \infty.  
\end{cases}$$

Introducing $\phi(r)$ as the smooth part of $\psi(r)$:  

$$\psi(r) = e^{-i\omega r_*} \phi(r),$$

the equation transforms to:

$$f(r)\phi''(r) + \left[f'(r) - 2i\omega\right]\phi'(r) - \tilde{V}(r) \phi(r) = 0.$$

where $\tilde{V}(r) = \frac{V(r)}{f(r)} $. To make coefficients of the equation finite for $r \in [0, +\infty)$, we define a new function

```math
F = \frac{\Delta}{r^4} \,.
```

Then, we have

```math
r^2 F(r) \phi''(r) + \left[r^2 F'(r)+2 r F(r)-2 i \omega \right] \phi'(r) - \tilde{V} \phi(r)  = 0 \,,
```

where,

```math
\tilde{V}(x) = (x-1) \left(\frac{l (l+1) (x-1)}{r_+^2}-F'(x)\right)+2 F(x) \,.
```

For numerical computation, compactify the radial coordinate using:

```math
x \equiv 1 - \frac{r_+}{r} \,.
```

Here, $x \in [0, 1]$. We can transform this equation into a generalized eigenvalue problem

```math
A \phi = \omega B \phi \,,
```

where

```math
\begin{align}
A &= \left[ (x-1)^2 F(x) \right] \frac{\partial^2}{\partial x^2} + \left[ (x-1)^2 F'(x) \right]  \frac{\partial}{\partial x} - \tilde{V} \,, \\
B &= \frac{2 i (x-1)^2}{r_+} \frac{\partial}{\partial x} \,.
\end{align}
```

To enforce the boundary condition at $\phi(x = 1) = 0$, we need to modify the last line of the matrix $B$ to $\mathcal{B} \phi = \mathbf{0}$, where

```math
\mathcal{B} = \left(\begin{array}{cccc}
T_0(1) & T_1(1) & T_2(1) & \cdots
\end{array}\right) = \left(\begin{array}{cccc}
1 & 1 & 1 & \cdots
\end{array}\right) \,.
```

In [38]:
l = 0
n = 40

# domain x ∈ [0, 1]
x_min = zero(TF)
x_max = one(TF)
dom = x_min .. x_max

chebSpace = Chebyshev(dom)
D = Derivative(chebSpace)
conversion1to2 = Conversion(Ultraspherical(1, dom), Ultraspherical(2, dom))

x = Fun(chebSpace)
F = (r₊ * (r₊^3 / R^2 + r₊ * (x - 1)^2 + 2 * M * (x - 1)^3) + Q^2 * (x - 1)^4) / r₊^4
dF = F'

# generalized eigenvalue problem A ϕ = ω B ϕ

# construct matrix A
cA2 = (x - 1)^2 * F
cA1 = (x - 1)^2 * dF
Vtilde = 2 * F + (x - 1) * (l * (l + 1) * (x - 1) / r₊^2 - dF)
A = cA2 * D^2 + cA1 * D - Vtilde
Am = @view(A[1:n, 1:n]) |> Matrix |> complex

# construct matrix B
cB1 = 2im * (x - 1)^2 / r₊
B = conversion1to2 * (cB1 * D)
Bm = @view(B[1:n, 1:n]) |> Matrix

# enforce Dirichlet boundary conditions at x = 1
Am[end, :] .= 1
Bm[end, :] .= 0

# Solve the generalized eigenvalue problem A ϕ = ω B ϕ
eigs = eigvals!(Am, Bm; sortby=abs)
qnm = filter(x -> !isnan(x) && imag(x) < 0, eigs)

39-element Vector{ComplexF64}:
      -2.798223242812926 - 2.6712058259179776im
      2.7982232428958094 - 2.6712058258583853im
       4.758489239969626 - 5.037569058024075im
      -4.758489271825203 - 5.037569088858869im
      -6.719272837541509 - 7.3944971365220304im
       6.719268963153064 - 7.394504117748148im
      -8.682064239957313 - 9.758728552649952im
       8.683195318522056 - 9.759466138411545im
       7.633774997106999 - 12.593343294902878im
      -7.608565160457379 - 12.613001756961257im
                         ⋮
       23.81936337976662 - 9.025580442952322im
  -0.0011010914714505429 - 25.808346826532258im
     -26.995926562284616 - 8.344057764969753im
       26.99592656215148 - 8.344057766689616im
       30.63761229251485 - 7.452225325553942im
     -30.637612292646807 - 7.452225325444598im
       35.12595504943825 - 6.177853340905617im
     -35.125955049453054 - 6.177853340903534im
 -3.5960151884096195e-10 - 490.5215474410868im

# Ensure you perform a convergence test before using the results; this notebook is just a **tutorial**!